# Web Scraping for Data with Python

## What is Web Scraping?

Pulling information and data from web sites. That's it!

## Why learn how to web scrape?

[![Items to scrape](https://drive.google.com/uc?export=view&id=1Dw6zLQ0nXul06fikIX_YzCR33-pDBLVX)](https://www.homechef.com/our-menu)

- To find information that isn't already contained somewhere
- To build your own data sets
- To quickly extract similar data from several different sources
- To solve niche questions that may not be answerable through publicly available data
- ... To feel like a cool hacker

The **power** of web scraping comes from how we can *automate* the process to get a whole bunch of data really quickly.

## What is HTML and CSS?

Press F12 on any web site. We'll use the Home Chef menu page.

[![Website](https://drive.google.com/uc?export=view&id=1OrY4IkjbXW9xVhrBdgHtmGEjSA5H8hK1)](https://www.homechef.com/our-menu)

*It's just glorified text.*

### HTML Elements vs. CSS Rules

![Elements vs Rules](https://htmlcss.learn.uno/html-and-css/css-selectors/css-selectors-1f0064.png)

[*From "HTML & CSS Is Hard"*](https://htmlcss.learn.uno/html-and-css/css-selectors/)

![HTML and CSS example](https://drive.google.com/uc?export=view&id=1UsuHlSYZjTSWo0Tb6WTaWglyPDyZhoqG)

[*From "HTML & CSS Is Hard"*](https://htmlcss.learn.uno/html-and-css/css-selectors/)

Going back to Home Chef...
This means we can change the font family for each item into something like Comic Sans MS.

![Comic Sans](https://drive.google.com/uc?export=view&id=17nsln_JKE1amVa4gGPSAlzWb2RKkOnMB)

The next time you hit F12 on a website, you might find something interesting...

[![Star Wars Source](https://drive.google.com/uc?export=view&id=1RkxyCPSKSFl2easXvn4cnjS0TmN1l78L)](https://www.starwars.com/)

[*From starwars.com*](https://www.starwars.com/)

### HUGE CAVEAT TO WEB SCRAPING

[![Site TOS](https://drive.google.com/uc?export=view&id=11gGA3w14ts6YhQXZnzA58bF0Duo35Bfp)](https://www.homechef.com/terms)

*From Home Chef's TOS*

**IF THE SITE SAYS DON'T SCRAPE, DON'T SCRAPE!**

## One method of grabbing a website's HTML text

*Using a website that we can actually scrape...*

[![Books to scrape](https://drive.google.com/uc?export=view&id=1qHmBVSIdHy9UUaFEof6ERGLe_TUu3b7N)](https://books.toscrape.com/)

In [1]:
# Import the Requests module from the Python Standard Library
import requests

# Website we will be scraping
url = "https://books.toscrape.com/"

# Get a response object from the web server
response = requests.get(url)
response.encoding = "UTF-8"

# Save HTML from url
html = response.text

# Print the HTML from the response object
print(html)

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

That is pretty disgusting to look at. How do we get the data we need?

## Using BeautifulSoup to parse HTML

![French Onion Soup](https://drive.google.com/uc?export=view&id=1eOq3TRcY1eEeDoVawduGMsLRCkL6CT6z)

BeautifulSoup is a library that helps us go through HTML and pick out the information we want.

In [2]:
# Import BeautifulSoup for parsing HTML
from bs4 import BeautifulSoup

# Parse HTML from the previous cell
soup = BeautifulSoup(html, "lxml")

# Counter to keep track of number of books iterated through
count = 1

# Dictionary to translate star ratings to numeric ratings
ratings = {
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5
}

# Get total number of pages listed on the home page (always has 50 pages)
num_pages = int(soup.select_one("li.current").text.split()[-1])

# For each count in the total number of pages...
for page in range(1, num_pages+1):

    # URL for each page
    page_url = f"https://books.toscrape.com/catalogue/page-{page}.html"
    
    # Grab this page's HTML
    page_response = requests.get(page_url)
    page_response.encoding = "UTF-8"
    page_html = page_response.text
    
    # Parse this page using beautifulsoup
    soup = BeautifulSoup(page_html, "lxml")

    # Grab each container that contains book information (product_pod)
    books = soup.select("article.product_pod")
    
    # For each book container in our list of containers...
    for book in books:
        
        # Print page count
        print(f"Page: {page}")

        # Print the current count value (tells us what number book we're on)
        # To get the FULL title, we target the title attribute stored in the
        # anchor tag, not the text visible in the h3 tag
        print(f"Book {count}: {book.h3.a['title']}")

        # Print the link (make into absolute links)
        print(f"Link: books.toscrape.com/catalogue/{book.a['href']}")
        
        # Print rating from class name
        num_stars = book.p["class"][1].lower()
        print(f"Rating (Number of Stars): {ratings[num_stars]}")

        # Print price of book (account for £ symbol)
        price = book.select_one("p.price_color").text.strip()[1:]
        print(f"Price (Pound Sterling): {price}")

        # Print availability of book
        status = book.select_one("p.availability").text.strip()
        print(f"Availability: {status}")

        # Quick and dirty line separator
        print("-" * 80)

        # Increment count to use for the next book in the loop
        count += 1

print("Scraping completed.")

Page: 1
Book 1: A Light in the Attic
Link: books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html
Rating (Number of Stars): 3
Price (Pound Sterling): 51.77
Availability: In stock
--------------------------------------------------------------------------------
Page: 1
Book 2: Tipping the Velvet
Link: books.toscrape.com/catalogue/tipping-the-velvet_999/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 53.74
Availability: In stock
--------------------------------------------------------------------------------
Page: 1
Book 3: Soumission
Link: books.toscrape.com/catalogue/soumission_998/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 50.10
Availability: In stock
--------------------------------------------------------------------------------
Page: 1
Book 4: Sharp Objects
Link: books.toscrape.com/catalogue/sharp-objects_997/index.html
Rating (Number of Stars): 4
Price (Pound Sterling): 47.82
Availability: In stock
--------------------------------------

Price (Pound Sterling): 26.41
Availability: In stock
--------------------------------------------------------------------------------
Page: 3
Book 43: Private Paris (Private #10)
Link: books.toscrape.com/catalogue/private-paris-private-10_958/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 47.61
Availability: In stock
--------------------------------------------------------------------------------
Page: 3
Book 44: #HigherSelfie: Wake Up Your Life. Free Your Soul. Find Your Tribe.
Link: books.toscrape.com/catalogue/higherselfie-wake-up-your-life-free-your-soul-find-your-tribe_957/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 23.11
Availability: In stock
--------------------------------------------------------------------------------
Page: 3
Book 45: Without Borders (Wanderlove #1)
Link: books.toscrape.com/catalogue/without-borders-wanderlove-1_956/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 45.07
Availability: In stock
-----------------

Price (Pound Sterling): 13.34
Availability: In stock
--------------------------------------------------------------------------------
Page: 5
Book 83: Pop Gun War, Volume 1: Gift
Link: books.toscrape.com/catalogue/pop-gun-war-volume-1-gift_918/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 18.97
Availability: In stock
--------------------------------------------------------------------------------
Page: 5
Book 84: Political Suicide: Missteps, Peccadilloes, Bad Calls, Backroom Hijinx, Sordid Pasts, Rotten Breaks, and Just Plain Dumb Mistakes in the Annals of American Politics
Link: books.toscrape.com/catalogue/political-suicide-missteps-peccadilloes-bad-calls-backroom-hijinx-sordid-pasts-rotten-breaks-and-just-plain-dumb-mistakes-in-the-annals-of-american-politics_917/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 36.28
Availability: In stock
--------------------------------------------------------------------------------
Page: 5
Book 85: Patience
Link: b

Price (Pound Sterling): 42.95
Availability: In stock
--------------------------------------------------------------------------------
Page: 7
Book 123: A Piece of Sky, a Grain of Rice: A Memoir in Four Meditations
Link: books.toscrape.com/catalogue/a-piece-of-sky-a-grain-of-rice-a-memoir-in-four-meditations_878/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 56.76
Availability: In stock
--------------------------------------------------------------------------------
Page: 7
Book 124: A Murder in Time
Link: books.toscrape.com/catalogue/a-murder-in-time_877/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 16.64
Availability: In stock
--------------------------------------------------------------------------------
Page: 7
Book 125: A Flight of Arrows (The Pathfinders #2)
Link: books.toscrape.com/catalogue/a-flight-of-arrows-the-pathfinders-2_876/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 55.53
Availability: In stock
------------------------

Availability: In stock
--------------------------------------------------------------------------------
Page: 9
Book 163: The Art of War
Link: books.toscrape.com/catalogue/the-art-of-war_838/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 33.34
Availability: In stock
--------------------------------------------------------------------------------
Page: 9
Book 164: The Argonauts
Link: books.toscrape.com/catalogue/the-argonauts_837/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 10.93
Availability: In stock
--------------------------------------------------------------------------------
Page: 9
Book 165: The 10% Entrepreneur: Live Your Startup Dream Without Quitting Your Day Job
Link: books.toscrape.com/catalogue/the-10-entrepreneur-live-your-startup-dream-without-quitting-your-day-job_836/index.html
Rating (Number of Stars): 3
Price (Pound Sterling): 27.55
Availability: In stock
-------------------------------------------------------------------------------

...

Book 789: Rework
Link: books.toscrape.com/catalogue/rework_212/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 44.88
Availability: In stock
--------------------------------------------------------------------------------
Page: 40
Book 790: Reservations for Two
Link: books.toscrape.com/catalogue/reservations-for-two_211/index.html
Rating (Number of Stars): 3
Price (Pound Sterling): 11.10
Availability: In stock
--------------------------------------------------------------------------------
Page: 40
Book 791: Red: The True Story of Red Riding Hood
Link: books.toscrape.com/catalogue/red-the-true-story-of-red-riding-hood_210/index.html
Rating (Number of Stars): 3
Price (Pound Sterling): 28.54
Availability: In stock
--------------------------------------------------------------------------------
Page: 40
Book 792: Ready Player One
Link: books.toscrape.com/catalogue/ready-player-one_209/index.html
Rating (Number of Stars): 4
Price (Pound Sterling): 19.07
Availability: In stock

Page: 42
Book 821: Jurassic Park (Jurassic Park #1)
Link: books.toscrape.com/catalogue/jurassic-park-jurassic-park-1_180/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 44.97
Availability: In stock
--------------------------------------------------------------------------------
Page: 42
Book 822: It's Never Too Late to Begin Again: Discovering Creativity and Meaning at Midlife and Beyond
Link: books.toscrape.com/catalogue/its-never-too-late-to-begin-again-discovering-creativity-and-meaning-at-midlife-and-beyond_179/index.html
Rating (Number of Stars): 1
Price (Pound Sterling): 42.38
Availability: In stock
--------------------------------------------------------------------------------
Page: 42
Book 823: Is Everyone Hanging Out Without Me? (And Other Concerns)
Link: books.toscrape.com/catalogue/is-everyone-hanging-out-without-me-and-other-concerns_178/index.html
Rating (Number of Stars): 3
Price (Pound Sterling): 20.11
Availability: In stock
------------------------------

Page: 45
Book 881: Annie on My Mind
Link: books.toscrape.com/catalogue/annie-on-my-mind_120/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 36.83
Availability: In stock
--------------------------------------------------------------------------------
Page: 45
Book 882: And Then There Were None
Link: books.toscrape.com/catalogue/and-then-there-were-none_119/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 35.01
Availability: In stock
--------------------------------------------------------------------------------
Page: 45
Book 883: A Walk in the Woods: Rediscovering America on the Appalachian Trail
Link: books.toscrape.com/catalogue/a-walk-in-the-woods-rediscovering-america-on-the-appalachian-trail_118/index.html
Rating (Number of Stars): 4
Price (Pound Sterling): 30.48
Availability: In stock
--------------------------------------------------------------------------------
Page: 45
Book 884: A Visit from the Goon Squad
Link: books.toscrape.com/catalogue/a-visi

Page: 47
Book 921: The Secret Healer
Link: books.toscrape.com/catalogue/the-secret-healer_80/index.html
Rating (Number of Stars): 3
Price (Pound Sterling): 34.56
Availability: In stock
--------------------------------------------------------------------------------
Page: 47
Book 922: The Sandman, Vol. 1: Preludes and Nocturnes (The Sandman (volumes) #1)
Link: books.toscrape.com/catalogue/the-sandman-vol-1-preludes-and-nocturnes-the-sandman-volumes-1_79/index.html
Rating (Number of Stars): 3
Price (Pound Sterling): 54.12
Availability: In stock
--------------------------------------------------------------------------------
Page: 47
Book 923: The Republic
Link: books.toscrape.com/catalogue/the-republic_78/index.html
Rating (Number of Stars): 3
Price (Pound Sterling): 33.78
Availability: In stock
--------------------------------------------------------------------------------
Page: 47
Book 924: The Odyssey
Link: books.toscrape.com/catalogue/the-odyssey_77/index.html
Rating (Number of Star

Availability: In stock
--------------------------------------------------------------------------------
Page: 49
Book 963: Off Sides (Off #1)
Link: books.toscrape.com/catalogue/off-sides-off-1_38/index.html
Rating (Number of Stars): 5
Price (Pound Sterling): 39.45
Availability: In stock
--------------------------------------------------------------------------------
Page: 49
Book 964: Of Mice and Men
Link: books.toscrape.com/catalogue/of-mice-and-men_37/index.html
Rating (Number of Stars): 2
Price (Pound Sterling): 47.11
Availability: In stock
--------------------------------------------------------------------------------
Page: 49
Book 965: Myriad (Prentor #1)
Link: books.toscrape.com/catalogue/myriad-prentor-1_36/index.html
Rating (Number of Stars): 4
Price (Pound Sterling): 58.75
Availability: In stock
--------------------------------------------------------------------------------
Page: 49
Book 966: My Perfect Mistake (Over the Top #1)
Link: books.toscrape.com/catalogue/my-perfect-

## But what if we need to interact with the web page?

![Interacting with the page](https://drive.google.com/uc?export=view&id=1QIy_k2Bzf1Mw5YaZIdajHW9-7p8icbh0)

There a solution for this!